In [126]:
# -*- coding:utf-8 -*-
import pandas as pd
import numpy as np
from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *
from urllib import quote_plus as urlquote

# 全量明细list

password = "Casstime@0719"
engine = create_engine('mysql+pymysql://ximu:%s@cassec-demo-read.mysql.rds.aliyuncs.com:3306/ec_admin'%urlquote(password))

password = "Casstime@com1112"
engine2 = create_engine('mysql+pymysql://reader:%s@172.21.21.78:3306/kscredit'%urlquote(password))

def get_sql(sql,engine):
    df = pd.read_sql(sql=sql,con = engine)
    #print df.shape
    #print df.columns
    return df

sql_str = """
SELECT * FROM dsbs_apply_gds_screen_record
"""
df_loan_rules = get_sql(sql_str,engine2)

df_loan_rules["dsbs_scrn_feedback_desc"] = df_loan_rules["dsbs_scrn_feedback_desc"].fillna("")
df1 = pd.concat([pd.Series(row[u'dsbs_apply_no'], row['dsbs_scrn_feedback_desc']\
                            .split(';')) for _, row in df_loan_rules.iterrows()]).reset_index()
df1.columns = ["rules","dsbs_apply_no"]

mask = df1["rules"].str.contains(":") & (df1["rules"].str.contains("外部数据异常") == False)  \
             & (df1["rules"].str.contains("未获取到数据") == False) \
            & (df1["rules"].str.contains("数据格式") == False) \
                | (df1["rules"]=="") 
df1_f = df1[mask].copy()
mask = df1["rules"].str.contains(":") & (df1["rules"].str.contains("外部数据异常") == False)  \
             & (df1["rules"].str.contains("未获取到数据") == False) \
            & (df1["rules"].str.contains("数据格式") == False) \
                & (df1["rules"].apply(lambda x:len(x))>0) 
df1_f = df1[mask].copy()

df1_f["hit"] = 1
df_piv_rule = df1_f.pivot_table(index="dsbs_apply_no",columns = "rules",values="hit",aggfunc="count")
df_piv_rule.columns = pd.MultiIndex.from_product([[u"规则触发"],df_piv_rule.columns])


sql_str = u"""
SELECT t4.plfm_order_no as `订单编号`,
t1.cust_code as `客户编号`,t2.enterprise_name as `客户`,t3.fclt_name as `产品`,
t1.dsbs_apply_no,apply_amt/100 AS `金额`,
apply_begin_time AS `申请时间`,
CASE WHEN apply_status IN ("WAIT_XIMU","WAIT_FDING") THEN "待审核"
WHEN apply_status LIKE "REJECT%%" THEN "拒绝"
WHEN apply_status LIKE "DONE" THEN "通过"
WHEN apply_status LIKE "CANCEL" THEN "取消"
ELSE "其他" END AS `状态`
FROM dsbs_apply_info t1
LEFT JOIN  `cust_base_info_e` t2
ON t1.cust_code = t2.cust_code
LEFT JOIN fclt_base_info t3
ON t1.fclt_code = t3.fclt_code
left join dsbs_apply_order_info t4
on t1.dsbs_apply_no = t4.dsbs_apply_no
"""
df_loan_apply = get_sql(sql_str,engine2)
df_loan_apply = df_loan_apply.set_index("dsbs_apply_no")
dc = df_loan_apply.copy()
df_loan_apply.columns = pd.MultiIndex.from_product([[u"支用基本信息"],df_loan_apply.columns])

df_loan_apply_m = pd.concat([df_loan_apply,df_piv_rule],axis = 1,sort = False)


df_loan_apply_m = df_loan_apply_m.sort_values(by = (u"支用基本信息",u"申请时间"),ascending = False)

In [188]:
df_loan_rules[df_loan_rules['dsbs_apply_no']=='CDNKSiKSiBL201912280397']


#E254160971507965953    神木市信德汇汽车维修服务店

id            dsbs_apply_no       dsbs_apply_id dsbs_scrn_status  \
24794  28146  CDNKSiKSiBL201912280397  262323094079676416         APPROVED   

      dsbs_scrn_feedback_time dsbs_scrn_feedback_result  \
24794              2019-12-28                      None   

             dsbs_scrn_feedback_desc  precdn_pay_scale  precdn_pay_amt  \
24794  WINI0001A1104:经营异常-近三个月交易笔数;;               0.0             0.0   

       bcard_score  bcard_score_weight  seller_deceiver_flag  warn_flag  \
24794       622.57                0.85                   NaN        NaN   

      warn_reason  memo  cr_user_id             cr_date  op_user_id  \
24794        None  None           1 2019-12-28 13:55:41           1   

                  op_date  delete_flag  
24794 2019-12-28 13:55:41            0

In [187]:
df_loan_rules

id            dsbs_apply_no       dsbs_apply_id dsbs_scrn_status  \
0       3352  CDNKSiKSiBL201911050001  242941160069279744    MANUALAPPROVE   
1       3353  CDNKSiKSiBL201911050001  242941160069279744         APPROVED   
2       3354  CDNKSiKSiBL201911050002  242942026721529856    MANUALAPPROVE   
3       3355  CDNKSiKSiBL201911050002  242942026721529856         APPROVED   
4       3356  CDNKSiKSiBL201911050003  242942478678757376    MANUALAPPROVE   
5       3357  CDNKSiKSiBL201911050003  242942478678757376         APPROVED   
6       3358  CDNKSiKSiBL201911050004  242943796390662144    MANUALAPPROVE   
7       3359  CDNKSiKSiBL201911050004  242943796390662144         APPROVED   
8       3360  CDNKSiKSiBL201911050005  242947751980044288    MANUALAPPROVE   
9       3361  CDNKSiKSiBL201911050006  242948397777031168    MANUALAPPROVE   
10      3362  CDNKSiKSiBL201911050006  242948397777031168         APPROVED   
11      3363  CDNKSiKSiBL201911050007  242948982744027136    MANUALAPPROVE   
12      3364  CDNKSiKSiBL201911050007  242948982744027136         APPROVED   
13      3365  CDNKSiKSiBL201911050008  242949382603804672    MANUALAPPROVE   
14      3366  CDNKSiKSiBL201911050008  242949382603804672         APPROVED   
15      3367  CDNKSiKSiBL201911050009  242949809516843008    MANUALAPPROVE   
16      3368  CDNKSiKSiBL201911050009  242949809516843008         APPROVED   
17      3369  CDNKSiKSiBL201911050010  242951081632468992    MANUALAPPROVE   
18      3370  CDNKSiKSiBL201911050010  242951081632468992         APPROVED   
19      3371  CDNKSiKSiBL201911050011  242952499344969728    MANUALAPPROVE   
20      3372  CDNKSiKSiBL201911050011  242952499344969728         APPROVED   
21      3373  CDNKSiKSiBL201911050012  242955641335844864    MANUALAPPROVE   
22      3374  CDNKSiKSiBL201911050013  242955647379836928    MANUALAPPROVE   
23      3375  CDNKSiKSiBL201911050014  242955808822792192    MANUALAPPROVE   
24      3376  CDNKSiKSiBL201911050013  242955647379836928         APPROVED   
25      3377  CDNKSiKSiBL201911050014  242955808822792192         APPROVED   
26      3378  CDNKSiKSiBL201911050015  242957044288589824    MANUALAPPROVE   
27      3379  CDNKSiKSiBL201911050015  242957044288589824         APPROVED   
28      3380  CDNKSiKSiBL201911050016  242958209780813824    MANUALAPPROVE   
29      3381  CDNKSiKSiBL201911050016  242958209780813824         APPROVED   
...      ...                      ...                 ...              ...   
87070  90422  CDNKSiKSiBL202004260119  305760381844897792         APPROVED   
87071  90423  CDNKSiKSiBL202004260120  305760393605726208         APPROVED   
87072  90424  CDNKSiKSiBL202004260121  305761026756849664         APPROVED   
87073  90425  CDNKSiKSiBL202004260122  305761096134828032         APPROVED   
87074  90426  CDNKSiKSiBL202004260123  305761275550384128         APPROVED   
87075  90427  CDNKSiKSiBL202004260124  305761295989231616         APPROVED   
87076  90428  CDNKSiKSiBL202004260125  305761440516562944         APPROVED   
87077  90429  CDNKSiKSiBL202004260126  305761538118021120         APPROVED   
87078  90430  CDNKSiKSiBL202004260127  305761764769796096         APPROVED   
87079  90431  CDNKSiKSiBL202004260128  305761867962265600         APPROVED   
87080  90432  CDNKSiKSiBL202004260129  305762070245167104         APPROVED   
87081  90433  CDNKSiKSiBL202004260130  305762211991666688         APPROVED   
87082  90434  CDNKSiKSiBL202004260131  305762375275921408         APPROVED   
87083  90435  CDNKSiKSiBL202004260132  305762604222029824         APPROVED   
87084  90436  CDNKSiKSiBL202004260133  305763015620300800         APPROVED   
87085  90437  CDNKSiKSiBL202004260134  305763035719442432         APPROVED   
87086  90438  CDNKSiKSiBL202004260135  305763241735266304         APPROVED   
87087  90439  CDNKSiKSiBL202004260136  305763385646002176         APPROVED   
87088  90440  CDNKSiKSiBL202004260137  305763448245993472         APPROVED   
87089  90441  CDNKSiKSiBL2020042

In [180]:

mask = df1["rules"].str.contains(":") & (df1["rules"].str.contains("外部数据异常") == False)  \
             & (df1["rules"].str.contains("未获取到数据") == False) \
            & (df1["rules"].str.contains("数据格式") == False) \
                | (df1["rules"]=="") 
df1_f = df1[mask].copy()

In [186]:
df1_f["hit"] = 1
df1_f[df1_f["rules"] == '']

rules            dsbs_apply_no  hit
82            CDNKSiKSiBL201911050023    1
96            CDNKSiKSiBL201911050029    1
209           CDNKSiKSiBL201911070022    1
233           CDNKSiKSiBL201911080005    1
246           CDNKSiKSiBL201911080015    1
282           CDNKSiKSiBL201911080027    1
327           CDNKSiKSiBL201911080044    1
335           CDNKSiKSiBL201911080049    1
395           CDNKSiKSiBL201911090005    1
396           CDNKSiKSiBL201911090006    1
409           CDNKSiKSiBL201911090012    1
451           CDNKSiKSiBL201911090029    1
461           CDNKSiKSiBL201911090035    1
462           CDNKSiKSiBL201911090036    1
475           CDNKSiKSiBL201911090044    1
480           CDNKSiKSiBL201911090047    1
485           CDNKSiKSiBL201911090050    1
520           CDNKSiKSiBL201911100020    1
561           CDNKSiKSiBL201911110005    1
578           CDNKSiKSiBL201911110012    1
580           CDNKSiKSiBL201911110013    1
590           CDNKSiKSiBL201911110019    1
591           CDNKSiKSiBL201911110020    1
596           CDNKSiKSiBL201911110024    1
597           CDNKSiKSiBL201911110025    1
598           CDNKSiKSiBL201911110026    1
599           CDNKSiKSiBL201911110027    1
602           CDNKSiKSiBL201911110029    1
603           CDNKSiKSiBL201911110030    1
610           CDNKSiKSiBL201911110034    1
...      ...                      ...  ...
157614        CDNKSiKSiBL202004260119    1
157616        CDNKSiKSiBL202004260120    1
157619        CDNKSiKSiBL202004260121    1
157622        CDNKSiKSiBL202004260122    1
157624        CDNKSiKSiBL202004260123    1
157626        CDNKSiKSiBL202004260124    1
157629        CDNKSiKSiBL202004260125    1
157631        CDNKSiKSiBL202004260126    1
157633        CDNKSiKSiBL202004260127    1
157636        CDNKSiKSiBL202004260128    1
157638        CDNKSiKSiBL202004260129    1
157640        CDNKSiKSiBL202004260130    1
157642        CDNKSiKSiBL202004260131    1
157644        CDNKSiKSiBL202004260132    1
157646        CDNKSiKSiBL202004260133    1
157648        CDNKSiKSiBL202004260134    1
157650        CDNKSiKSiBL202004260135    1
157653        CDNKSiKSiBL202004260136    1
157656        CDNKSiKSiBL202004260137    1
157658        CDNKSiKSiBL202004260138    1
157661        CDNKSiKSiBL202004260139    1
157664        CDNKSiKSiBL202004260140    1
157666        CDNKSiKSiBL202004260141    1
157668        CDNKSiKSiBL202004260142    1
157670        CDNKSiKSiBL202004260143    1
157672        CDNKSiKSiBL202004260144    1
157674        CDNKSiKSiBL202004260145    1
157677        CDNKSiKSiBL202004260146    1
157679        CDNKSiKSiBL202004260147    1
157681        CDNKSiKSiBL202004260148    1

[85077 rows x 3 columns]

In [176]:
df1_f.shape

(148175, 2)

In [177]:
mask = df1["rules"].str.contains(":") & (df1["rules"].str.contains("外部数据异常") == False)  \
             & (df1["rules"].str.contains("未获取到数据") == False) \
            & (df1["rules"].str.contains("数据格式") == False) \
                & (df1["rules"].apply(lambda x:len(x))>0) 
df1_f1 = df1[mask].copy()
df1_f1.shape

(63098, 2)

In [178]:
df1[df1['rules']==''].shape

(85077, 2)

In [179]:
85077+63098

148175

In [ ]:
mask

In [ ]:
sql_str = """
SELECT * FROM dsbs_apply_gds_screen_record
"""
df_loan_rules = get_sql(sql_str,engine2)

In [147]:
mask = df1["rules"].str.contains(":") & (df1["rules"].str.contains("外部数据异常") == False)  \
             & (df1["rules"].str.contains("未获取到数据") == False) \
            & (df1["rules"].str.contains("数据格式") == False) \
                & (df1["rules"].apply(lambda x:len(x))>0) 
df1_f = df1[mask].copy()

In [149]:
df1_f["hit"] = 1
df_piv_rule = df1_f.pivot_table(index="dsbs_apply_no",columns = "rules",values="hit",aggfunc="count")
df_piv_rule.columns = pd.MultiIndex.from_product([[u"规则触发"],df_piv_rule.columns])

In [150]:
sql_str = u"""
SELECT t4.plfm_order_no as `订单编号`,
t1.cust_code as `客户编号`,t2.enterprise_name as `客户`,t3.fclt_name as `产品`,
t1.dsbs_apply_no,apply_amt/100 AS `金额`,
apply_begin_time AS `申请时间`,
CASE WHEN apply_status IN ("WAIT_XIMU","WAIT_FDING") THEN "待审核"
WHEN apply_status LIKE "REJECT%%" THEN "拒绝"
WHEN apply_status LIKE "DONE" THEN "通过"
WHEN apply_status LIKE "CANCEL" THEN "取消"
ELSE "其他" END AS `状态`
FROM dsbs_apply_info t1
LEFT JOIN  `cust_base_info_e` t2
ON t1.cust_code = t2.cust_code
LEFT JOIN fclt_base_info t3
ON t1.fclt_code = t3.fclt_code
left join dsbs_apply_order_info t4
on t1.dsbs_apply_no = t4.dsbs_apply_no
"""
df_loan_apply = get_sql(sql_str,engine2)
df_loan_apply = df_loan_apply.set_index("dsbs_apply_no")
dc = df_loan_apply.copy()
df_loan_apply.columns = pd.MultiIndex.from_product([[u"支用基本信息"],df_loan_apply.columns])

df_loan_apply_m1 = pd.concat([df_loan_apply,df_piv_rule],axis = 1,sort = False)


df_loan_apply_m1 = df_loan_apply_m1.sort_values(by = (u"支用基本信息",u"申请时间"),ascending = False)

In [155]:
df_loan_apply_m1.columns = df_loan_apply_m1.columns.droplevel()
df_loan_apply_m1

订单编号                 客户编号  \
CDNKSiKSiBL202004260180  S2004260001127  E246064614863802369   
CDNKSiKSiBL202004260179  S2004260001708  E254217607236816897   
CDNKSiKSiBL202004260178  S2004260001774  E258370387044208641   
CDNKSiKSiBL202004260177  S2004260001963  E293919657918775297   
CDNKSiKSiBL202004260176  S2004260001715  E305090106325221377   
CDNKSiKSiBL202004260175  S2004260001981  E250339777751027713   
CDNKSiKSiBL202004260174  S2004260001160  E302545682831523841   
CDNKSiKSiBL202004260173  S2004260001474  E252161422853672961   
CDNKSiKSiBL202004260172  S2004260001910  E254218249930018817   
CDNKSiKSiBL202004260171  S2004260001689  E254134025067700225   
CDNKSiKSiBL202004260170  S2004260001358  E254166733907042305   
CDNKSiKSiBL202004260169  S2004260001158  E248940687234437121   
CDNKSiKSiBL202004260168  S2004260001995  E254187521473257473   
CDNKSiKSiBL202004260167  S2004260001584  E252213154078003201   
CDNKSiKSiBL202004260166  S2004260001571  E305090106325221377   
CDNKSiKSiBL202004260165  S2004260001501  E293919657918775297   
CDNKSiKSiBL202004260164  S2004260001462  E255868630405746689   
CDNKSiKSiBL202004260163  S2004260001325  E293919657918775297   
CDNKSiKSiBL202004260162  S2004260001476  E293919657918775297   
CDNKSiKSiBL202004260161  S2004260001003  E254196219247595521   
CDNKSiKSiBL202004260160  S2004260001151  E246077915245056001   
CDNKSiKSiBL202004260159  S2004260001892  E254179701604487169   
CDNKSiKSiBL202004260158  S2004260001010  E301115523339444225   
CDNKSiKSiBL202004260157  S2004260001137  E298975894867816449   
CDNKSiKSiBL202004260156  S2004260001552  E254213922528169985   
CDNKSiKSiBL202004260155  S2004260001592  E255466352918138881   
CDNKSiKSiBL202004260154  S2004260001646  E254148419222773761   
CDNKSiKSiBL202004260153  S2004260001967  E254148419222773761   
CDNKSiKSiBL202004260152  S2004260001628  E254201169633611777   
CDNKSiKSiBL202004260151  S2004260001723  E301115523339444225   
...                                 ...                  ...   
CDNKSiKSiBL201911060002  S1911060001180  E243429585054601217   
CDNKSiKSiBL201911050029  S1911050006295  E243146480297185281   
CDNKSiKSiBL201911050028  S1911050006352  E243152705378324481   
CDNKSiKSiBL201911050026  S1911050006183  E243131298107887617   
CDNKSiKSiBL201911050027  S1911050006557  E243168701241626625   
CDNKSiKSiBL201911050024  S1911050005046  E243117329678864385   
CDNKSiKSiBL201911050025  S1911050005059  E243155335353733121   
CDNKSiKSiBL201911050021  S1911050004905  E243123947376218113   
CDNKSiKSiBL201911050022  S1911050004482  E243123947376218113   
CDNKSiKSiBL201911050023  S1911050004962  E243141161919713281   
CDNKSiKSiBL201911050019  S1911050003191  E243090465308545025   
CDNKSiKSiBL201911050020  S1911050003383  E243124715227451393   
CDNKSiKSiBL201911050018  S1911050000121  E242886946739720193   
CDNKSiKSiBL201911050017  S1911050000249  E242886946739720193   
CDNKSiKSiBL201911050016  S1911050000663  E242886946739720193   
CDNKSiKSiBL201911050015  S1911050000920  E242886946739720193   
CDNKSiKSiBL201911050014  S1911050000593  E242892153695506433   
CDNKSiKSiBL201911050013  S1911050000598  E242886946739720193   
CDNKSiKSiBL201911050012  S1911050000734  E242892153695506433   
CDNKSiKSiBL201911050011  S1911050000220  E242886946739720193   
CDNKSiKSiBL201911050010  S1911050000062  E242886946739720193   
CDNKSiKSiBL201911050009  S1911050000704  E242886946739720193   
CDNKSiKSiBL201911050008  S1911050000007  E242886946739720193   
CDNKSiKSiBL201911050007  S1911050000682  E242886946739720193   
CDNKSiKSiBL201911050006  S1911050000651  E242886946739720193   
CDNKSiKSiBL201911050005  S1911050000961  E242892153695506433   
CDNKSiKSiBL201911050004  S1911050000418  E242892153695506433   
CDNKSiKSiBL201911050003  S1911050000894  E242892153695506433   
CDNKSiKSiBL201911050002  S1911050000629  E242892153695506433   
CDNKSiKSiBL201911050001  S1911050000749  E242892153695506433   

                                          客户    产品       金额  \
CDN

In [165]:
df_loan_apply_m1[df_loan_apply_m1[u'客户编号']=='E254160971507965953'][u'金额'].sum()

329075.28

In [152]:
df_loan_apply.shape

(86229, 7)

In [133]:
df_loan_rules["dsbs_scrn_feedback_desc"] = df_loan_rules["dsbs_scrn_feedback_desc"].fillna("")
df1 = pd.concat([pd.Series(row[u'dsbs_apply_no'], row['dsbs_scrn_feedback_desc']\
                            .split(';')) for _, row in df_loan_rules.iterrows()]).reset_index()
df1.columns = ["rules","dsbs_apply_no"]

In [127]:
df_loan_rules.shape

(87100, 20)

In [136]:
df1[df1["rules"]==""].shape

(85077, 2)

In [134]:
df1.shape

(157682, 2)

In [131]:
df1_f.shape

(148175, 3)

In [132]:
df1.head()

rules            dsbs_apply_no
0       DINI0001A1100:经营异常-最近交易时间  CDNKSiKSiBL201911050001
1     DINI0001A1103:经营异常-近三个月交易笔数  CDNKSiKSiBL201911050001
2  DINI0001A1105:经营异常-近三个月交易金额增长率  CDNKSiKSiBL201911050001
3                            test  CDNKSiKSiBL201911050001
4       DINI0001A1100:经营异常-最近交易时间  CDNKSiKSiBL201911050002

In [118]:
df_loan_apply_m = pd.concat([df_loan_apply,df_piv_rule],axis = 1,sort = False)


df_loan_apply_m = df_loan_apply_m.sort_values(by = (u"支用基本信息",u"申请时间"),ascending = False)

In [119]:
# 触发率统计
df_loan_apply_m.columns = df_loan_apply_m.columns.droplevel()
df_loan_apply_m

订单编号                 客户编号  \
CDNKSiKSiBL202004240726  S2004240006864  E257921109511376897   
CDNKSiKSiBL202004240725  S2004240005704  E254215815782469633   
CDNKSiKSiBL202004240724  S2004240006316  E283361123598602241   
CDNKSiKSiBL202004240723  S2004240006842  E258027138001670145   
CDNKSiKSiBL202004240722  S2004240006854  E254125375330721793   
CDNKSiKSiBL202004240721  S2004240006051  E254125375330721793   
CDNKSiKSiBL202004240720  S2004240006443  E248940687234437121   
CDNKSiKSiBL202004240719  S2004240006626  E260920108329734145   
CDNKSiKSiBL202004240718  S2004240006657  E301456428550365185   
CDNKSiKSiBL202004240717  S2004240006460  E256217910786134017   
CDNKSiKSiBL202004240716  S2004240006526  E256564086693695489   
CDNKSiKSiBL202004240715  S2004240006835  E256564086693695489   
CDNKSiKSiBL202004240714  S2004240006957  E256564086693695489   
CDNKSiKSiBL202004240713  S2004240006547  E302573008088764417   
CDNKSiKSiBL202004240712  S2004240006438  E305079774663946241   
CDNKSiKSiBL202004240711  S2004240006663  E253638365595832321   
CDNKSiKSiBL202004240710  S2004240006463  E256143302674354177   
CDNKSiKSiBL202004240709  S2004240006333  E254691015099682817   
CDNKSiKSiBL202004240708  S2004240006461  E243543806148874241   
CDNKSiKSiBL202004240707  S2004240006644  E254691015099682817   
CDNKSiKSiBL202004240706  S2004240006228  E291242945489694721   
CDNKSiKSiBL202004240705  S2004240004014  E253638365595832321   
CDNKSiKSiBL202004240704  S2004240006261  E292065150721118209   
CDNKSiKSiBL202004240703  S2004240006847  E305120993528033281   
CDNKSiKSiBL202004240702  S2004240006142  E254203082332704769   
CDNKSiKSiBL202004240701  S2004240006381  E299648024513101825   
CDNKSiKSiBL202004240700  S2004240006121  E246407595714154497   
CDNKSiKSiBL202004240699  S2004240006285  E259436101956014081   
CDNKSiKSiBL202004240698  S2004240006247  E265997915497959425   
CDNKSiKSiBL202004240697  S2004240006383  E259375532624318465   
...                                 ...                  ...   
CDNKSiKSiBL201911060002  S1911060001180  E243429585054601217   
CDNKSiKSiBL201911050029  S1911050006295  E243146480297185281   
CDNKSiKSiBL201911050028  S1911050006352  E243152705378324481   
CDNKSiKSiBL201911050026  S1911050006183  E243131298107887617   
CDNKSiKSiBL201911050027  S1911050006557  E243168701241626625   
CDNKSiKSiBL201911050024  S1911050005046  E243117329678864385   
CDNKSiKSiBL201911050025  S1911050005059  E243155335353733121   
CDNKSiKSiBL201911050021  S1911050004905  E243123947376218113   
CDNKSiKSiBL201911050022  S1911050004482  E243123947376218113   
CDNKSiKSiBL201911050023  S1911050004962  E243141161919713281   
CDNKSiKSiBL201911050019  S1911050003191  E243090465308545025   
CDNKSiKSiBL201911050020  S1911050003383  E243124715227451393   
CDNKSiKSiBL201911050018  S1911050000121  E242886946739720193   
CDNKSiKSiBL201911050017  S1911050000249  E242886946739720193   
CDNKSiKSiBL201911050016  S1911050000663  E242886946739720193   
CDNKSiKSiBL201911050015  S1911050000920  E242886946739720193   
CDNKSiKSiBL201911050014  S1911050000593  E242892153695506433   
CDNKSiKSiBL201911050013  S1911050000598  E242886946739720193   
CDNKSiKSiBL201911050012  S1911050000734  E242892153695506433   
CDNKSiKSiBL201911050011  S1911050000220  E242886946739720193   
CDNKSiKSiBL201911050010  S1911050000062  E242886946739720193   
CDNKSiKSiBL201911050009  S1911050000704  E242886946739720193   
CDNKSiKSiBL201911050008  S1911050000007  E242886946739720193   
CDNKSiKSiBL201911050007  S1911050000682  E242886946739720193   
CDNKSiKSiBL201911050006  S1911050000651  E242886946739720193   
CDNKSiKSiBL201911050005  S1911050000961  E242892153695506433   
CDNKSiKSiBL201911050004  S1911050000418  E242892153695506433   
CDNKSiKSiBL201911050003  S1911050000894  E242892153695506433   
CDNKSiKSiBL201911050002  S1911050000629  E242892153695506433   
CDNKSiKSiBL201911050001  S1911050000749  E242892153695506433   

                                          客户    产品        金额  \
CD

In [120]:
col1 = list('col_'+(str(i)) for i in range(len(df_loan_apply_m.columns)))
dic = dict(map(lambda x,y:[x,y],df_loan_apply_m.columns,col1))
dic2 = dict(map(lambda x,y:[x,y],col1,df_loan_apply_m.columns))
df_loan_apply_m1 = df_loan_apply_m.rename(columns=dic).copy()

In [121]:
df_1 = df_loan_apply_m1.groupby(['col_1','col_3'])['col_2','col_4'].agg({'col_2':np.min,'col_4':np.sum})
df_2 = df_loan_apply_m1.groupby(['col_1','col_3'])[col1[9:]].sum()

In [125]:
df_1.merge(df_2,on=['col_1','col_3'],how='left').reset_index().rename(columns=dic2).to_excel(u'./规则触碰.xlsx')

In [ ]:
# import datetime
# dt = datetime.date.today()
# dt_str = dt.strftime("%Y-%m-%d")
# df_loan_apply_m.to_excel(u"D:/casstime_ana/10_手工报表/R02_支用规则触发/开思_支用规则触发_"+dt_str+".xlsx")
# df_loan_apply_m.to_excel(u"//10.118.71.218/徙木临时文件/R02_支用规则触发/开思_支用规则触发_"+dt_str+".xlsx")

##---------------------------------------------------------------------------------------------------##
# 触发率统计
df_loan_apply_m.columns = df_loan_apply_m.columns.droplevel()

tt = df_loan_apply_m.copy()
tt = tt.fillna(0)
tt['date'] = tt[u"申请时间"].dt.date
tt['date_str'] = tt[u"申请时间"].apply(lambda x:str(x))
tt['month'] = tt['date'].apply(lambda x:str(x)[:7])
tt=tt.reset_index()
tt["key"]=tt["index"]+tt[u"订单编号"]+tt[u"客户编号"]+tt["date_str"]
tt.index = tt["key"]

col = tt.columns.tolist()
print(len(col))
col = col[9:]
col.remove('date')
col.remove('month')
col.remove('date_str')
col.remove('key')
print(len(col))

ts = tt[col]
ts = ts.stack().reset_index().rename({"rules":"value"},axis = 1)
ts.columns=['key','rule','value']

tt= tt.drop('key',1)
tt=tt.reset_index()

ds = pd.merge(ts,tt,on = 'key',how = 'left')
 
ds['occ'] = 'casstime'
ds['customer'] = ds[u"客户编号"]
ds['dsbs_apply_no'] = ds[u"订单编号"]

order = ds.groupby(['month','date','occ']).agg({'key':pd.Series.nunique})
order = pd.DataFrame(order)
order = order.reset_index()

ss = ds.copy()
ss = ss[ss['value'] == 1]
# ss['value']=ss['value'].apply(lambda x:x if x == 1 else None)

result = ss.groupby(['month','date','occ','rule']).agg({'value':'sum','customer':pd.Series.nunique,'dsbs_apply_no':pd.Series.nunique})
result = pd.DataFrame(result)
result = result.reset_index()

total = pd.merge(result,order,on = ['month','date','occ'],how = 'left')
total['hit_ratio'] = np.round(total['value']/total['key'],4)

c = ['month','date','occ','rule','value','customer','dsbs_apply_no','hit_ratio']
ttl = total.loc[:,c]
ttl = ttl.sort_values(by='date',ascending = False)
ttl.columns = [u'统计月份',u'统计日期',u'场景',u'规则',u'触发次数',u'触发客户数',u'触发订单',u'触发率']

# ttl.to_excel(u"D:/casstime_ana/10_手工报表/R02_支用规则触发/开思_支用规则触发_统计报表_"+dt_str+".xlsx")
# ttl.to_excel(u"//10.118.71.218/徙木临时文件/R02_支用规则触发/开思_支用规则触发_统计报表_"+dt_str+".xlsx")

In [23]:
# -*- coding:utf-8 -*-
import numpy as np
import pandas as pd

from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *
from urllib import quote_plus as urlquote

password = "Casstime@0719"
engine = create_engine('mysql+pymysql://ximu:%s@cassec-demo-read.mysql.rds.aliyuncs.com:3306/ec_admin'%urlquote(password))

password = "Casstime@com1112"
engine2 = create_engine('mysql+pymysql://reader:%s@172.21.21.78:3306/kscredit'%urlquote(password))

def get_sql(sql,engine):
    df = pd.read_sql(sql=sql,con = engine)
    print df.shape
    print df.columns
    return df


sql = """-- 逾期
SELECT cust_code,
COUNT(1) AS delq_loan_cnt,
SUM(loan_bal)/100 AS delq_loan_bal,
MIN(loan_date) AS fst_delq_loan_date,
MIN(end_date) AS fst_delq_end_date
FROM kscredit_corebanking.ac_view_co_loan_bal
WHERE over_day > 0 AND loan_bal >0
GROUP BY cust_code;"""

df_delq_cust = get_sql(sql,engine2)

sql = """-- 余额
SELECT cust_code,
COUNT(1) AS bal_loan_cnt,
SUM(loan_bal)/100 AS loan_bal
FROM kscredit_corebanking.ac_view_co_loan_bal
WHERE loan_bal >0
GROUP BY cust_code;"""
df_bal_cust = get_sql(sql,engine2)

sql = """
SELECT t1.cust_code,t2.ett_mem_code,t3.cust_name 
FROM cust_member_info t1
LEFT JOIN cust_xm_ett_mem_rel t2
ON t1.mem_code = t2.xm_mem_code
LEFT JOIN cust_base_info t3
ON t1.cust_code = t3.cust_code
"""
df_cust_info = get_sql(sql,engine2)

sql = """
SELECT 
xm_list_code AS 徙木订单号,
cust_code 客户编号,
t2.product_name 产品,
loan_date 贷款日,
end_date 到期日,
loan_amt/100 AS 本金金额,
loan_bal/100 AS 本金余额,
over_day 逾期天数
FROM kscredit_corebanking.ac_view_co_loan_bal t1
LEFT JOIN kscredit_corebanking.product_base_info t2
ON t1.prod_code = t2.product_code
WHERE over_day > 0 AND loan_bal >0;
"""
df_delq_order = get_sql(sql,engine2)


df_merge = df_cust_info.merge(df_delq_cust,on = "cust_code",how="right").merge(df_bal_cust,on = "cust_code",how = "left")
df_merge = df_merge.rename(columns = {"cust_code":u"客户编号",
                                      "ett_mem_code":u"平台编号",
                                      "cust_name":u"客户名称",
                           "delq_loan_cnt":u"逾期订单数",
                           "delq_loan_bal":u"逾期余额",
                           "fst_delq_loan_date":u"逾期订单最早借款日",
                           "fst_delq_end_date":u"逾期订单最早到期日",
                           "bal_loan_cnt":u"余额订单数",
                           "loan_bal":u"余额"}).sort_values(by = u"逾期余额",ascending = False)

import datetime
dt = datetime.date.today()
dt_str = dt.strftime("%Y-%m-%d")
writer = pd.ExcelWriter(u'D:/casstime_ana/10_手工报表/R01_逾期明细/开思时代_逾期明细_new%s.xlsx'%dt_str)
df_merge.to_excel(writer,sheet_name=u'逾期客户',index=False)
df_delq_order.to_excel(writer,sheet_name=u'逾期订单',index=False)
writer.save()
writer1 = pd.ExcelWriter(u'//10.118.71.218/徙木临时文件/R01_逾期明细/开思时代_逾期明细_%s.xlsx'%dt_str)
df_merge.to_excel(writer1,sheet_name=u'逾期客户',index=False)
df_delq_order.to_excel(writer1,sheet_name=u'逾期订单',index=False)
print('done')
writer1.save()


(440, 5)
Index([u'cust_code', u'delq_loan_cnt', u'delq_loan_bal', u'fst_delq_loan_date',
       u'fst_delq_end_date'],
      dtype='object')
(2048, 3)
Index([u'cust_code', u'bal_loan_cnt', u'loan_bal'], dtype='object')
(3469, 3)
Index([u'cust_code', u'ett_mem_code', u'cust_name'], dtype='object')
(1723, 8)
Index([u'徙木订单号', u'客户编号', u'产品', u'贷款日', u'到期日', u'本金金额', u'本金余额', u'逾期天数'], dtype='object')
done


In [81]:
df1_f

rules            dsbs_apply_no  hit
0            DINI0001A1100:经营异常-最近交易时间  CDNKSiKSiBL201911050001    1
1          DINI0001A1103:经营异常-近三个月交易笔数  CDNKSiKSiBL201911050001    1
2       DINI0001A1105:经营异常-近三个月交易金额增长率  CDNKSiKSiBL201911050001    1
4            DINI0001A1100:经营异常-最近交易时间  CDNKSiKSiBL201911050002    1
5          DINI0001A1103:经营异常-近三个月交易笔数  CDNKSiKSiBL201911050002    1
6       DINI0001A1105:经营异常-近三个月交易金额增长率  CDNKSiKSiBL201911050002    1
8            DINI0001A1100:经营异常-最近交易时间  CDNKSiKSiBL201911050003    1
9          DINI0001A1103:经营异常-近三个月交易笔数  CDNKSiKSiBL201911050003    1
10      DINI0001A1105:经营异常-近三个月交易金额增长率  CDNKSiKSiBL201911050003    1
12           DINI0001A1100:经营异常-最近交易时间  CDNKSiKSiBL201911050004    1
13         DINI0001A1103:经营异常-近三个月交易笔数  CDNKSiKSiBL201911050004    1
14      DINI0001A1105:经营异常-近三个月交易金额增长率  CDNKSiKSiBL201911050004    1
16           DINI0001A1100:经营异常-最近交易时间  CDNKSiKSiBL201911050005    1
17         DINI0001A1103:经营异常-近三个月交易笔数  CDNKSiKSiBL201911050005    1
18      DINI0001A1105:经营异常-近三个月交易金额增长率  CDNKSiKSiBL201911050005    1
19           DINI0001A1100:经营异常-最近交易时间  CDNKSiKSiBL201911050006    1
20         DINI0001A1103:经营异常-近三个月交易笔数  CDNKSiKSiBL201911050006    1
21      DINI0001A1105:经营异常-近三个月交易金额增长率  CDNKSiKSiBL201911050006    1
23           DINI0001A1100:经营异常-最近交易时间  CDNKSiKSiBL201911050007    1
24         DINI0001A1103:经营异常-近三个月交易笔数  CDNKSiKSiBL201911050007    1
25      DINI0001A1105:经营异常-近三个月交易金额增长率  CDNKSiKSiBL201911050007    1
27           DINI0001A1100:经营异常-最近交易时间  CDNKSiKSiBL201911050008    1
28         DINI0001A1103:经营异常-近三个月交易笔数  CDNKSiKSiBL201911050008    1
29      DINI0001A1105:经营异常-近三个月交易金额增长率  CDNKSiKSiBL201911050008    1
31           DINI0001A1100:经营异常-最近交易时间  CDNKSiKSiBL201911050009    1
32         DINI0001A1103:经营异常-近三个月交易笔数  CDNKSiKSiBL201911050009    1
33      DINI0001A1105:经营异常-近三个月交易金额增长率  CDNKSiKSiBL201911050009    1
35           DINI0001A1100:经营异常-最近交易时间  CDNKSiKSiBL201911050010    1
36         DINI0001A1103:经营异常-近三个月交易笔数  CDNKSiKSiBL201911050010    1
37      DINI0001A1105:经营异常-近三个月交易金额增长率  CDNKSiKSiBL201911050010    1
...                                ...                      ...  ...
140648                                  CDNKSiKSiBL202004170175    1
140649       WINI0001A1101:经营异常-最近交易时间  CDNKSiKSiBL202004170176    1
140650                                  CDNKSiKSiBL202004170176    1
140651       WINI0001A1101:经营异常-最近交易时间  CDNKSiKSiBL202004170177    1
140652                                  CDNKSiKSiBL202004170177    1
140653       WINI0001A1101:经营异常-最近交易时间  CDNKSiKSiBL202004170178    1
140654                                  CDNKSiKSiBL202004170178    1
140655       WINI0001A1101:经营异常-最近交易时间  CDNKSiKSiBL202004170179    1
140656                                  CDNKSiKSiBL202004170179    1
140657       WINI0001A1101:经营异常-最近交易时间  CDNKSiKSiBL202004170180    1
140658     WINI0001A1104:经营异常-近三个月交易笔数  CDNKSiKSiBL202004170180    1
140659                                  CDNKSiKSiBL202004170180    1
140660       WINI0001A1101:经营异常-最近交易时间  CDNKSiKSiBL202004170181    1
140661                                  CDNKSiKSiBL202004170181    1
140662       WINI0001A1101:经营异常-最近交易时间  CDNKSiKSiBL202004170182    1
140663                                  CDNKSiKSiBL202004170182    1
140664       WINI0001A1101:经营异常-最近交易时间  CDNKSiKSiBL202004170183    1
140665                                  CDNKSiKSiBL202004170183    1
140666       WINI0001A1101:经营异常-最近交易时间  CDNKSiKSiBL202004170184    1
140667                                  CDNKSiKSiBL202004170184    1
140668       WINI0001A1101:经营异常-最近交易时间  CDNKSiKSiBL202004170185    1
140669                                  CDNKSiKSiBL202004170185    1
140670       WINI0001A1101:经营异常-最近交易时间  CDNKSiKSiBL202004170186    1
140671                                  CDNKSiKSiBL202004170186    1
140672       WINI0001A1101:经营异常-最近交易时间  CDNKSiKSiBL202004170187    1
140673                                  CDNKSiKSiBL202004170187    1
140674       WINI0001A1101:经营异常

In [5]:
# -*- coding:utf-8 -*-
import pandas as pd
import numpy as np
from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *
from urllib import quote_plus as urlquote

# 全量明细list

password = "Casstime@0719"
engine = create_engine('mysql+pymysql://ximu:%s@cassec-demo-read.mysql.rds.aliyuncs.com:3306/ec_admin'%urlquote(password))

password = "Casstime@com1112"
engine2 = create_engine('mysql+pymysql://reader:%s@172.21.21.78:3306/kscredit'%urlquote(password))

def get_sql(sql,engine):
    df = pd.read_sql(sql=sql,con = engine)
    return df

sql_str = """
SELECT * FROM dsbs_apply_gds_screen_record
"""
df_loan_rules = get_sql(sql_str,engine2)

df_loan_rules["dsbs_scrn_feedback_desc"] = df_loan_rules["dsbs_scrn_feedback_desc"].fillna("")
df1 = pd.concat([pd.Series(row[u'dsbs_apply_no'], row['dsbs_scrn_feedback_desc']\
                            .split(';')) for _, row in df_loan_rules.iterrows()]).reset_index()
df1.columns = ["rules","dsbs_apply_no"]

mask = df1["rules"].str.contains(":") & (df1["rules"].str.contains("外部数据异常") == False)  \
             & (df1["rules"].str.contains("未获取到数据") == False) \
            & (df1["rules"].str.contains("数据格式") == False) \
                | (df1["rules"]=="") 
df1_f = df1[mask].copy()


df1_f["hit"] = 1
df_piv_rule = df1_f.pivot_table(index="dsbs_apply_no",columns = "rules",values="hit",aggfunc="count")
df_piv_rule.columns = pd.MultiIndex.from_product([[u"规则触发"],df_piv_rule.columns])


sql_str = u"""
SELECT t4.plfm_order_no AS `订单编号`,
t1.cust_code AS `客户编号`,t2.enterprise_name AS `客户`,t3.fclt_name AS `产品`,
t2.registered_date AS `企业注册时间`,t2.business_addr_other AS `企业经营地址`,
t2.province AS `所在省`,t1.lat AS `授信额度`,
t1.dsbs_apply_no,apply_amt/100 AS `金额`,
apply_begin_time AS `申请时间`,
CASE WHEN apply_status IN ("WAIT_XIMU","WAIT_FDING") THEN "待审核"
WHEN apply_status LIKE "REJECT%%" THEN "拒绝"
WHEN apply_status LIKE "DONE" THEN "通过"
WHEN apply_status LIKE "CANCEL" THEN "取消"
ELSE "其他" END AS `状态`
FROM dsbs_apply_info t1
LEFT JOIN  `cust_base_info_e` t2
ON t1.cust_code = t2.cust_code
LEFT JOIN fclt_base_info t3
ON t1.fclt_code = t3.fclt_code
LEFT JOIN dsbs_apply_order_info t4
ON t1.dsbs_apply_no = t4.dsbs_apply_no
"""
df_loan_apply = get_sql(sql_str,engine2)
df_loan_apply = df_loan_apply.set_index("dsbs_apply_no")
dc = df_loan_apply.copy()
df_loan_apply.columns = pd.MultiIndex.from_product([[u"支用基本信息"],df_loan_apply.columns])

df_loan_apply_m = pd.concat([df_loan_apply,df_piv_rule],axis = 1,sort = False)
#df_loan_apply_m.shape

df_loan_apply_m = df_loan_apply_m.sort_values(by = (u"支用基本信息",u"申请时间"),ascending = False)
# import datetime
# dt = datetime.date.today()
# dt_str = dt.strftime("%Y-%m-%d")
# df_loan_apply_m.to_excel(u"D:/casstime_ana/10_手工报表/R02_支用规则触发/开思_支用规则触发_"+dt_str+".xlsx")
# df_loan_apply_m.to_excel(u"//10.118.71.218/徙木临时文件/R02_支用规则触发/开思_支用规则触发_"+dt_str+".xlsx")

In [2]:
df1_f.head()

rules            dsbs_apply_no  hit
0       DINI0001A1100:经营异常-最近交易时间  CDNKSiKSiBL201911050001    1
1     DINI0001A1103:经营异常-近三个月交易笔数  CDNKSiKSiBL201911050001    1
2  DINI0001A1105:经营异常-近三个月交易金额增长率  CDNKSiKSiBL201911050001    1
4       DINI0001A1100:经营异常-最近交易时间  CDNKSiKSiBL201911050002    1
5     DINI0001A1103:经营异常-近三个月交易笔数  CDNKSiKSiBL201911050002    1

In [3]:
df_piv_rule.head()

规则触发                                 \
                             DINI0001A1003:首次支用-支用时间-授信时间未知   
dsbs_apply_no                                                 
CDNKSiKSiBL201911050001  NaN                            NaN   
CDNKSiKSiBL201911050002  NaN                            NaN   
CDNKSiKSiBL201911050003  NaN                            NaN   
CDNKSiKSiBL201911050004  NaN                            NaN   
CDNKSiKSiBL201911050005  NaN                            NaN   

                                                                             \
                        DINI0001A1100:经营异常-最近交易时间 DINI0001A1101:经营异常-最近交易时间   
dsbs_apply_no                                                                 
CDNKSiKSiBL201911050001                       1.0                       NaN   
CDNKSiKSiBL201911050002                       1.0                       NaN   
CDNKSiKSiBL201911050003                       1.0                       NaN   
CDNKSiKSiBL201911050004                       1.0                       NaN   
CDNKSiKSiBL201911050005                       1.0                       NaN   

                                                                               \
                        DINI0001A1102:经营异常-最近交易时间 DINI0001A1103:经营异常-近三个月交易笔数   
dsbs_apply_no                                                                   
CDNKSiKSiBL201911050001                       NaN                         1.0   
CDNKSiKSiBL201911050002                       NaN                         1.0   
CDNKSiKSiBL201911050003                       NaN                         1.0   
CDNKSiKSiBL201911050004                       NaN                         1.0   
CDNKSiKSiBL201911050005                       NaN                         1.0   

                                                     \
                        DINI0001A1104:经营异常-近三个月交易笔数   
dsbs_apply_no                                         
CDNKSiKSiBL201911050001                         NaN   
CDNKSiKSiBL201911050002                         NaN   
CDNKSiKSiBL201911050003                         NaN   
CDNKSiKSiBL201911050004                         NaN   
CDNKSiKSiBL201911050005                         NaN   

                                                        \
                        DINI0001A1105:经营异常-近三个月交易金额增长率   
dsbs_apply_no                                            
CDNKSiKSiBL201911050001                            1.0   
CDNKSiKSiBL201911050002                            1.0   
CDNKSiKSiBL201911050003                            1.0   
CDNKSiKSiBL201911050004                            1.0   
CDNKSiKSiBL201911050005                            1.0   

                                                                             \
                        DTPG0001A1101:注册资本金小于30万 DTPG0001A1102:注册资本金大于1000万   
dsbs_apply_no                                                                 
CDNKSiKSiBL201911050001                      NaN                        NaN   
CDNKSiKSiBL201911050002                      NaN                        NaN   
CDNKSiKSiBL201911050003                      NaN                        NaN   
CDNKSiKSiBL201911050004                      NaN                        NaN   
CDNKSiKSiBL201911050005                      NaN                        NaN   

                         ...                             \
                         ... WTPG0001A1102:注册资本金大于1000万   
dsbs_apply_no            ...                              
CDNKSiKSiBL201911050001  ...                        NaN   
CDNKSiKSiBL201911050002  ...                        NaN   
CDNKSiKSiBL201911050003  ...                        NaN   
CDNKSiKSiBL201911050004  ...                        NaN   
CDNKSiKSiBL201911050005  ...                        NaN   

                                                                          \
                        WTPG0001A1104:成立年限小于3年 WTPG0001A1105:营业结束日期在1年以内   
dsbs_apply_no                                                            

In [ ]:
df_piv_rule.meger(df_loan_apply)

In [50]:
df_loan_apply

支用基本信息                       \
                                   订单编号                 客户编号   
dsbs_apply_no                                                  
CDNKSiKSiBL201911050006  S1911050000651  E242886946739720193   
CDNKSiKSiBL201911050007  S1911050000682  E242886946739720193   
CDNKSiKSiBL201911050008  S1911050000007  E242886946739720193   
CDNKSiKSiBL201911050009  S1911050000704  E242886946739720193   
CDNKSiKSiBL201911050010  S1911050000062  E242886946739720193   
CDNKSiKSiBL201911050011  S1911050000220  E242886946739720193   
CDNKSiKSiBL201911050013  S1911050000598  E242886946739720193   
CDNKSiKSiBL201911050015  S1911050000920  E242886946739720193   
CDNKSiKSiBL201911050016  S1911050000663  E242886946739720193   
CDNKSiKSiBL201911050017  S1911050000249  E242886946739720193   
CDNKSiKSiBL201911050018  S1911050000121  E242886946739720193   
CDNKSiKSiBL201911070024  S1911070007389  E242886946739720193   
CDNKSiKSiBL201911070025  S1911070007286  E242886946739720193   
CDNKSiKSiBL201911070026  S1911070007996  E242886946739720193   
CDNKSiKSiBL201911080001  S1911080000305  E242886946739720193   
CDNKSiKSiBL201911080002  S1911080000477  E242886946739720193   
CDNKSiKSiBL201911050001  S1911050000749  E242892153695506433   
CDNKSiKSiBL201911050002  S1911050000629  E242892153695506433   
CDNKSiKSiBL201911050003  S1911050000894  E242892153695506433   
CDNKSiKSiBL201911050004  S1911050000418  E242892153695506433   
CDNKSiKSiBL201911050005  S1911050000961  E242892153695506433   
CDNKSiKSiBL201911050012  S1911050000734  E242892153695506433   
CDNKSiKSiBL201911050014  S1911050000593  E242892153695506433   
CDNKSiKSiBL201911070027  S1911070007748  E242892153695506433   
CDNKSiKSiBL201911080003  S1911080000293  E242892153695506433   
CDNKSiKSiBL201911050019  S1911050003191  E243090465308545025   
CDNKSiKSiBL201911070015  S1911070006332  E243090465308545025   
CDNKSiKSiBL201911070016  S1911070006957  E243090465308545025   
CDNKSiKSiBL201911120024  S1911120001058  E243090465308545025   
CDNKSiKSiBL201911270116  S1911270003866  E243090465308545025   
...                                 ...                  ...   
CDNKSiKSiBL202004160867  S2004160007594  E301521189765955585   
CDNKSiKSiBL202004160084  S2004160000178  E301523412847722497   
CDNKSiKSiBL202004160085  S2004160000428  E301523412847722497   
CDNKSiKSiBL202004160122  S2004160001122  E301523412847722497   
CDNKSiKSiBL202004170056  S2004170000337  E301523412847722497   
CDNKSiKSiBL202004160050  S2004160000580  E301542743430156289   
CDNKSiKSiBL202004160052  S2004160000392  E301542743430156289   
CDNKSiKSiBL202004160056  S2004160000008  E301542743430156289   
CDNKSiKSiBL202004160066  S2004160000362  E301542743430156289   
CDNKSiKSiBL202004160785  S2004160007472  E301542743430156289   
CDNKSiKSiBL202004160812  S2004160007061  E301542743430156289   
CDNKSiKSiBL202004150754  S2004150006362  E301771571096240129   
CDNKSiKSiBL202004160405  S2004160003069  E301771571096240129   
CDNKSiKSiBL202004150529  S2004150004696  E301818446184722433   
CDNKSiKSiBL202004150530  S2004150004523  E301818446184722433   
CDNKSiKSiBL202004150531  S2004150004978  E301818446184722433   
CDNKSiKSiBL202004150596  S2004150003223  E301853734651060225   
CDNKSiKSiBL202004150600  S2004150005783  E301853734651060225   
CDNKSiKSiBL202004160652  S2004160005315  E301860480954179585   
CDNKSiKSiBL202004150626  S2004150005006  E301861783864385537   
CDNKSiKSiBL202004150688  S2004150006732  E301866288500723713   
CDNKSiKSiBL202004150689  S2004150006888  E301866288500723713   
CDNKSiKSiBL202004160526  S2004160004069  E301866288500723713   
CDNKSiKSiBL202004160482  S2004160004175  E302137887749578753   
CDNKSiKSiBL202004160261  S2004160002919  E302163275607457793   
CDNKSiKSiBL202004160615  S2004160005122  E302163275607457793   
CDNKSiKSiBL202004160649  S2004160005158  E302194265793019905   
CDNKSiKSiBL202004170090  S2004170000181  E302208524342026241   
CDNKSiKSiBL202004170091  S2004170000659  E302208524342026241   
C

In [26]:
df_loan_apply.to_excel('./kk.xlsx')

In [66]:
df = pd.read_excel('./df_kkk.xlsx')

In [68]:
df.head(1)

col_1                col_2            col_3 col_4 col_5 col_6  \
0  S2004170001872  E254163189564973057  深圳市罗湖区深华通汽车服务中心  开思挂账   NaN   NaN   

  col_7    col_8               col_9 col_10  ...  col_40  col_41  col_42  \
0   NaN  5000000 2020-04-17 11:01:31    待审核  ...     NaN     NaN     NaN   

   col_43  col_44  col_45  col_46  col_47  col_48  col_49  
0     NaN     NaN     NaN     NaN     NaN     NaN     NaN  

[1 rows x 49 columns]

In [80]:
df['col_11']

0        0
1        0
2        0
3        0
4        1
5        1
6        1
7        1
8        1
9        1
10       1
11       1
12       1
13       2
14       1
15       1
16       1
17       1
18       0
19       1
20       1
21       1
22       1
23       0
24       0
25       1
26       1
27       2
28       1
29       1
        ..
78587    1
78588    0
78589    2
78590    2
78591    1
78592    2
78593    1
78594    3
78595    3
78596    0
78597    1
78598    3
78599    3
78600    3
78601    3
78602    3
78603    3
78604    3
78605    3
78606    3
78607    3
78608    3
78609    3
78610    3
78611    3
78612    3
78613    3
78614    3
78615    3
78616    3
Name: col_11, Length: 78617, dtype: int64

In [76]:
col = list(df.columns)
coll = col[11:]
df.groupby([u'col_2',u'col_4'])[[u'col_11', u'col_12', u'col_13',
       u'col_14', u'col_15', u'col_16', u'col_17', u'col_18', u'col_19',
       u'col_20', u'col_21', u'col_22', u'col_23', u'col_24', u'col_25',
       u'col_26', u'col_27', u'col_28', u'col_29', u'col_30', u'col_31',
       u'col_32', u'col_33', u'col_34', u'col_35', u'col_36', u'col_37',
       u'col_38', u'col_39', u'col_40', u'col_41', u'col_42', u'col_43',
       u'col_44', u'col_45', u'col_46', u'col_47', u'col_48', u'col_49']].sum().reset_index().to_excel('./guize.xlsx')

In [79]:
df.groupby([u'col_2',u'col_4'])[u'col_1', u'col_3',u'col_5', u'col_8']\
            .agg({u'col_1':pd.Series.count, u'col_3':np.min,u'col_5':np.min, u'col_8':np.max})\
.reset_index().to_excel('./guize2.xlsx')

In [40]:
df.columns

Index([u'订单编号', u'客户编号', u'客户', u'产品', u'企业注册时间', u'企业经营地址', u'所在省', u'授信额度',
       u'申请时间', u'状态', u'触碰次数', u'DINI0001A1003:首次支用-支用时间-授信时间未知',
       u'DINI0001A1100:经营异常-最近交易时间', u'DINI0001A1101:经营异常-最近交易时间',
       u'DINI0001A1102:经营异常-最近交易时间', u'DINI0001A1103:经营异常-近三个月交易笔数',
       u'DINI0001A1104:经营异常-近三个月交易笔数', u'DINI0001A1105:经营异常-近三个月交易金额增长率',
       u'DTPG0001A1101:注册资本金小于30万', u'DTPG0001A1102:注册资本金大于1000万',
       u'DTPG0001A1104:成立年限小于3年', u'DTPG0001A1105:营业结束日期在1年以内',
       u'DTPG0001A1107:法定代表人占股百分二十以下', u'DTPG0001A1108:股东数量大于等于5',
       u'DTPG0001A1110:近1年发生章程变更', u'DTPG0001A1111:法律公告信息',
       u'DTPG0001A1112:近2年有涉诉记录（被告）', u'DTPG0001A1113:开庭公告信息',
       u'DTPG0001A1118:行政处罚', u'JMII0001A0001:申请异常-订单数据',
       u'NTPG0001A1103:成立年限小于1年', u'NTPG0001A1106:企业状态异常',
       u'NTPG0001A1114:历史被执行', u'NTPG0001A1115:历史失信', u'NTPG0001A1117:现行经营异常',
       u'WINI0001A1100:经营异常-最近交易时间', u'WINI0001A1101:经营异常-最近交易时间',
       u'WINI0001A1104:经营异常-近三个月交易笔数', u'WTPG0001A1101:注册资本金小

In [ ]:
##---------------------------------------------------------------------------------------------------##
# 触发率统计
df_loan_apply_m.columns = df_loan_apply_m.columns.droplevel()

tt = df_loan_apply_m.copy()
tt = tt.fillna(0)
tt['date'] = tt[u"申请时间"].dt.date
tt['date_str'] = tt[u"申请时间"].apply(lambda x:str(x))
tt['month'] = tt['date'].apply(lambda x:str(x)[:7])
tt=tt.reset_index()
tt["key"]=tt["index"]+tt[u"订单编号"]+tt[u"客户编号"]+tt["date_str"]
tt.index = tt["key"]

col = tt.columns.tolist()
print(len(col))
col = col[9:]
col.remove('date')
col.remove('month')
col.remove('date_str')
col.remove('key')
print(len(col))

ts = tt[col]
ts = ts.stack().reset_index().rename({"rules":"value"},axis = 1)
ts.columns=['key','rule','value']

tt= tt.drop('key',1)
tt=tt.reset_index()

ds = pd.merge(ts,tt,on = 'key',how = 'left')
 
ds['occ'] = 'casstime'
ds['customer'] = ds[u"客户编号"]
ds['dsbs_apply_no'] = ds[u"订单编号"]

order = ds.groupby(['month','date','occ']).agg({'key':pd.Series.nunique})
order = pd.DataFrame(order)
order = order.reset_index()

ss = ds.copy()
ss = ss[ss['value'] == 1]
# ss['value']=ss['value'].apply(lambda x:x if x == 1 else None)

result = ss.groupby(['month','date','occ','rule']).agg({'value':'sum','customer':pd.Series.nunique,'dsbs_apply_no':pd.Series.nunique})
result = pd.DataFrame(result)
result = result.reset_index()

total = pd.merge(result,order,on = ['month','date','occ'],how = 'left')
total['hit_ratio'] = np.round(total['value']/total['key'],4)

c = ['month','date','occ','rule','value','customer','dsbs_apply_no','hit_ratio']
ttl = total.loc[:,c]
ttl = ttl.sort_values(by='date',ascending = False)
ttl.columns = [u'统计月份',u'统计日期',u'场景',u'规则',u'触发次数',u'触发客户数',u'触发订单',u'触发率']

ttl.to_excel(u"D:/casstime_ana/10_手工报表/R02_支用规则触发/开思_支用规则触发_统计报表_"+dt_str+".xlsx")
ttl.to_excel(u"//10.118.71.218/徙木临时文件/R02_支用规则触发/开思_支用规则触发_统计报表_"+dt_str+".xlsx")